In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download the image assets from drive

In [ ]:
# Background Images
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/eva/eva5/week15/bg  /content/
# Foreground Images
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/eva/eva5/week15/fg  /content/
# Foreground Masks
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/eva/eva5/week15/fg_mask  /content/

## Overlay FG over BG (Generate FG_BG images and it's Masks)

In [ ]:
from random import randint
import cv2
import numpy as np

def makeCombinations( background, foreground, fg_mask, opDir, imageNo, labelFile, labelInfo):

  ratio = 1.0
  bg_wid, bg_hei = background.size
  for j in range(4):
    fg_wid, fg_hei = foreground.size
    fg_wid  = fg_wid * ratio
    fg_hei  = fg_hei * ratio

    #print("bw {} bh {} fw {} fh {}".format(bg_wid,bg_hei, fg_wid,fg_hei))
    fg_resized   = foreground.resize((int(fg_wid),int(fg_hei)), Image.ANTIALIAS)
    mask_resized = fg_mask.resize((int(fg_wid),int(fg_hei)), Image.ANTIALIAS)
    for i in range(0,5):
      labelData = labelInfo.copy()
      bg_copy = background.copy()
      
      pos_x = np.random.randint(0, max((bg_wid - fg_wid),0.1*bg_wid), size=1)
      pos_y = np.random.randint(0, max((bg_hei - fg_hei),0.1*bg_hei), size=1)
      bg_copy.paste(fg_resized, (pos_x, pos_y), fg_resized)
      
      bg_mask = Image.new('L', bg_copy.size)
      fg_mask = fg_mask.convert('L')
      bg_mask.paste(mask_resized, (pos_x, pos_y), mask_resized)
      # save the background and the mask as temp .jpg files    
      imageNo = imageNo + 1
      bg_copy.save(opDir+'/overlay/'+str(imageNo)+".jpg", quality=50, optimize=True)
      bg_mask.save(opDir+'/mask/'+str(imageNo)+".jpg", quality=50, optimize=True)

      labelData.append(opDir+'/overlay/'+str(imageNo)+".jpg")
      labelData.append(opDir+'/mask/'+str(imageNo)+".jpg")
      labelData.append(opDir+'/depth/'+str(imageNo)+".jpg")
      labelFile.write(labelData)

    ratio = ratio - 0.2
    

## CSVWriter class to create the Label Data

In [ ]:
import csv
class CSVWriter():

    filename = None
    fp = None
    writer = None

    def __init__(self, filename):
        self.filename = filename
        self.fp = open(self.filename, 'w', encoding='utf8')
        self.writer = csv.writer(self.fp, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL, lineterminator='\n')

    def close(self):
        self.fp.close()

    def write(self, elems):
        self.writer.writerow(elems)

    def size(self):
        return os.path.getsize(self.filename)

    def fname(self):
        return self.filename

## Create Overlay and Mask ( Main program)

In [ ]:
!rm -r Dataset

rm: cannot remove 'Dataset': No such file or directory


In [ ]:
from os import listdir

# from google.colab.patches import cv2_imshow
# import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import time

%matplotlib inline
import os, errno

# Paths used
DB_name = 'Dataset/'
path_BG = 'bg'
path_FG = 'fg'
path_fg_mask = 'fg_mask'
bg_imageListDir = listdir(path_BG)
fg_imageListDir = listdir(path_FG)

try:
    os.makedirs(DB_name + "bg")
except FileExistsError:
    pass

# All dataset info will save to "label_data.csv"
dataList = CSVWriter(DB_name + 'Dataset/label_data.csv')

program_starts = time.time()
for bgID,bg_image in enumerate(bg_imageListDir):
  now = time.time()
  print("iter = {}, bg_mage = {}, {} seconds since the loop started".format(bgID,bg_image, (now - program_starts)))
  outputDir = DB_name
  outputDir = outputDir + bg_image[:-4]
  bg_img = Image.open(path_BG+'/'+bg_image).resize((224,224), Image.ANTIALIAS)
  bg_img.save( DB_name + "bg/" + bg_image, quality=60, optimize=True)
  labelInfo = [DB_name + "bg/" + bg_image]
  for fg_image in fg_imageListDir:
    outputDir_fg = outputDir + '/' + fg_image[:-4]
    fg_img  = Image.open(path_FG+'/'+fg_image)
    #print("path_fg_mask{}/mask_fg_image{}".format(path_fg_mask,fg_image))
    fg_mask = Image.open(path_fg_mask+'/mask-'+fg_image)

    try:
        os.makedirs(outputDir_fg + "/overlay")
        os.makedirs(outputDir_fg + "/mask")
        os.makedirs(outputDir_fg + "/depth")
    except FileExistsError:
        pass
    
    makeCombinations( bg_img, fg_img, fg_mask, outputDir_fg, 0, dataList, labelInfo)
    # Flip foreground image and its mask horizontally
    fg_img = fg_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    fg_mask = fg_mask.transpose(PIL.Image.FLIP_LEFT_RIGHT)

    makeCombinations( bg_img, fg_img, fg_mask, outputDir_fg, 20, dataList, labelInfo)
print("{} seconds since the loop started".format((now - program_starts)))
dataList.close()


iter = 0, bg_mage = image-172.jpg, 0.0001857280731201172 seconds since the loop started
iter = 1, bg_mage = image-157.jpg, 12.938524961471558 seconds since the loop started
iter = 2, bg_mage = image-200.jpg, 27.85464644432068 seconds since the loop started
iter = 3, bg_mage = image-185.jpg, 42.45527172088623 seconds since the loop started
iter = 4, bg_mage = image-148.jpg, 56.12466216087341 seconds since the loop started
iter = 5, bg_mage = image-174.jpg, 70.75373554229736 seconds since the loop started
iter = 6, bg_mage = image-159.jpg, 84.71268224716187 seconds since the loop started
iter = 7, bg_mage = image-176.jpg, 100.50514006614685 seconds since the loop started
iter = 8, bg_mage = image-197.jpg, 115.18235683441162 seconds since the loop started
iter = 9, bg_mage = image-135.jpg, 129.39059257507324 seconds since the loop started
iter = 10, bg_mage = image-127.jpg, 144.12246584892273 seconds since the loop started
iter = 11, bg_mage = image-153.jpg, 158.92994713783264 seconds sin

### Compress Dataset folder to zip format

In [ ]:
import shutil
shutil.make_archive('Dataset', 'zip', 'Dataset')
#! unzip -q zip_file_name.zip

'/content/Dataset.zip'

### Copy compressed Dataset.zip file to Google drive

In [ ]:
!cp /content/Dataset.zip /content/drive/My\ Drive/Colab\ Notebooks/eva/eva5/week15/Outputzip